PART 1:NER INFERENCE PART

In [ ]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

#!pip install -q datasets seqeval
import transformers
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from datasets import Dataset
import numpy as np
from seqeval.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score
import torch
import pandas as pd
import re   

print("Transformers Version:", transformers.__version__)
print("CUDA Available?", torch.cuda.is_available())
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

import pandas as pd

# ✅ Load your dataset
main_df=pd.read_csv("D:\ML\grok_adddress_parser\grok_address_parser-env\exp1.csv")#path to your dataset
main_df['CUSTADDR'] = main_df['CUSTADDR'].str.lower()
print(main_df.head())

d:\conda_envs\address_cluster_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Transformers Version: 4.54.0
CUDA Available? True


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch


# ✅ Load the saved model + tokenizer 
model_path = "D:\ML\grok_adddress_parser\grok_address_parser-env\muril_120k_5epochs"#path to ner model folder
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)
from transformers import pipeline
ner_pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple",device=0)  # 'simple' groups tokens into entities

# Create label mappings
label_list1=['B-APARTMENT', 'B-AREA', 'B-BLOCK', 'B-FLOOR', 'B-HOUSE_NUMBER', 'B-KHASRA_NUMBER', 'B-LANDMARK', 'B-LOCALITY', 'B-LOCALITY2', 'B-PINCODE', 'B-PLOT_NUMBER', 
             'B-ROAD_DETAILS', 'B-VILLAGE', 'I-APARTMENT', 'I-AREA', 'I-BLOCK', 'I-FLOOR', 'I-HOUSE_NUMBER', 'I-KHASRA_NUMBER', 'I-LANDMARK', 'I-LOCALITY', 'I-LOCALITY2',
             'I-PLOT_NUMBER', 'I-ROAD_DETAILS', 'I-VILLAGE', 'O']
#label_list = sorted(set(label for row in df["labels"] for label in row.split()))
label2id = {label: i for i, label in enumerate(label_list1)}
id2label = {i: label for label, i in label2id.items()}
NUM_LABELS = len(label_list1)

print("\n",label_list1)
print("\n",id2label)
print("\n",label2id)

print("\n",NUM_LABELS)
# Map string tags to ID list
#df["label_ids"] = df["labels"].apply(lambda x: [label2id[l] for l in x.split()])
#print(df[["text", "labels", "label_ids"]].head(1))
#df.head()

print(id2label)
model.config.id2label = id2label
model.config.label2id = {v: k for k, v in id2label.items()}


Device set to use cuda:0



 ['B-APARTMENT', 'B-AREA', 'B-BLOCK', 'B-FLOOR', 'B-HOUSE_NUMBER', 'B-KHASRA_NUMBER', 'B-LANDMARK', 'B-LOCALITY', 'B-LOCALITY2', 'B-PINCODE', 'B-PLOT_NUMBER', 'B-ROAD_DETAILS', 'B-VILLAGE', 'I-APARTMENT', 'I-AREA', 'I-BLOCK', 'I-FLOOR', 'I-HOUSE_NUMBER', 'I-KHASRA_NUMBER', 'I-LANDMARK', 'I-LOCALITY', 'I-LOCALITY2', 'I-PLOT_NUMBER', 'I-ROAD_DETAILS', 'I-VILLAGE', 'O']

 {0: 'B-APARTMENT', 1: 'B-AREA', 2: 'B-BLOCK', 3: 'B-FLOOR', 4: 'B-HOUSE_NUMBER', 5: 'B-KHASRA_NUMBER', 6: 'B-LANDMARK', 7: 'B-LOCALITY', 8: 'B-LOCALITY2', 9: 'B-PINCODE', 10: 'B-PLOT_NUMBER', 11: 'B-ROAD_DETAILS', 12: 'B-VILLAGE', 13: 'I-APARTMENT', 14: 'I-AREA', 15: 'I-BLOCK', 16: 'I-FLOOR', 17: 'I-HOUSE_NUMBER', 18: 'I-KHASRA_NUMBER', 19: 'I-LANDMARK', 20: 'I-LOCALITY', 21: 'I-LOCALITY2', 22: 'I-PLOT_NUMBER', 23: 'I-ROAD_DETAILS', 24: 'I-VILLAGE', 25: 'O'}

 {'B-APARTMENT': 0, 'B-AREA': 1, 'B-BLOCK': 2, 'B-FLOOR': 3, 'B-HOUSE_NUMBER': 4, 'B-KHASRA_NUMBER': 5, 'B-LANDMARK': 6, 'B-LOCALITY': 7, 'B-LOCALITY2': 8, 'B-PINC

NEXT CELL DOES BASIC NORMALIZATION ON ADDRESSES AND SAVE IT IN A NEW COLOMN 'ADDR'

In [ ]:
import re

def standardize_address_abbreviations_enhanced(address_text):
    """
    Standardizes common abbreviations in an an address string, with enhanced
    handling for house numbers where abbreviations might be merged with digits.
    """
    if not isinstance(address_text, str):
        return address_text # Return as is if not a string (e.g., None, NaN)

    address_text = address_text.lower() # Convert to lowercase for consistent matching

    # --- NEW Step 0: General Cleaning and Space Introduction ---

    # 0.1: Replace hyphens and slashes with spaces
    address_text = re.sub(r'[-/]', ' ', address_text)

    # 0.2: Introduce a space when switching from an alphabet to a number (e.g., "A123" -> "A 123")
    address_text = re.sub(r'([a-z])(\d)', r'\1 \2', address_text)

    # 0.3: Introduce a space when switching from a number to an alphabet (e.g., "123A" -> "123 A")
    address_text = re.sub(r'(\d)([a-z])', r'\1 \2', address_text)

    # 0.4: Normalize multiple spaces to a single space, and strip leading/trailing spaces
    address_text = re.sub(r'\s+', ' ', address_text).strip()

    # --- Previous Step 1: Pre-process specific merged abbreviations (now less critical but still useful) ---
    # These specific regex patterns might still catch nuanced cases not covered by generic alpha-numeric split.
    # House Number patterns
    address_text = re.sub(r'\b(h|house|hn|hno|no|prop no|flat no|unit no|bldg no|room no|ro)\s*(\d+[a-z]*)\b', r'\1 \2', address_text)
    address_text = re.sub(r'(no|h no|h\.no|h-no)\s*(\d+([a-z])?)', r'\1 \2', address_text)
    address_text = re.sub(r'(house no|hse no)\s*(\d+([a-z])?)', r'\1 \2', address_text)
    address_text = re.sub(r'(prop no)\s*(\d+([a-z])?)', r'\1 \2', address_text)
    address_text = re.sub(r'(bldg no)\s*(\d+([a-z])?)', r'\1 \2', address_text)
    address_text = re.sub(r'(flat no)\s*(\d+([a-z])?)', r'\1 \2', address_text)
    address_text = re.sub(r'(room no|ro)\s*(\d+([a-z])?)', r'\1 \2', address_text)


    # Gali Number patterns
    address_text = re.sub(r'\b(g|gali|gali no|g no)\s*(\d+[a-z]*)\b', r'\1 \2', address_text)
    address_text = re.sub(r'(gali no|g no)\s*(\d+([a-z])?)', r'\1 \2', address_text)
    address_text = re.sub(r'(gali|g)\s*(\d+([a-z])?)', r'\1 \2', address_text)

    # --- Step 2: Define and apply comprehensive word-boundary based replacements ---
    FUZZY_MAP = {
        r'\bhn\b': 'house number',
        r'\bh no\b': 'house number',
        r'\bhouse no\b': 'house number',
        r'\bhse no\b': 'house number',
        r'\bhs no\b': 'house number',
        r'\bhno\b': 'house number',
        r'\bprop no\b': 'property number',
        r'\bflat no\b': 'flat number',
        r'\bunit no\b': 'unit number',
        r'\bkh no\b': 'khasra number',
        r'\bkhasra no\b': 'khasra number',
        r'\bkhno\b': 'khasra number',
        r'\bkhn\b': 'khasra number',


        r'\bg no\b': 'gali number',
        r'\bgali no\b': 'gali number',
        r'\bgno\b': 'gali number',
        r'\bgn\b': 'gali number',

        r'\bpno\b': 'plot number',

        r'\bextn\b': 'extension',
        r'\bext\b': 'extension',
        r'\bextnti\b': 'extension',
        

        r'\bngr\b': 'nagar',
        r'\bk ngr\b': 'karawal nagar',
        r'\blakshmi\b': 'laxmi',
        r'\bvill\b': 'village',


        r'\bph\b': 'phase',
        r'\bphs\b': 'phase',
        r'\bphase no\b': 'phase number',

       
        r'\brm\b': 'room',
        r'\broom no\b': 'room number',


        r'\brod\b': 'road',
        r'\bmarg\b': 'road',
       

        r'\bldg\b': 'building',
        r'\bbldg no\b': 'building number',
        r'\bblk\b': 'block',
        r'\bbl\b': 'block',

        r'\bapp\b': 'apartment',
        r'\bapt\b': 'apartment',
        r'\bfl\b': 'floor',
        r'\bg f\b': 'ground floor',
        r'\bf f\b': 'first floor',
        r'\bii f\b': 'second floor',
        r'\biii f\b': 'third floor',

        r'\blc\b': 'locality',
        r'\barea\b': 'area',
        r'\bpincode\b': 'pincode',
        r'\bpin code\b': 'pincode',

        r'\bnr\b': 'near',
        r'\bopp\b': 'opposite',
        r'\bop\b': 'opposite',
        r'\b(no)\b': 'number', # General 'no' at the end, if not caught by specific patterns
    }    # Sort keys by length (descending) to ensure longer, more specific matches are attempted first.
    sorted_fuzzy_map_keys = sorted(FUZZY_MAP.keys(), key=len, reverse=True)

    for abbr_pattern in sorted_fuzzy_map_keys:
        full_form = FUZZY_MAP[abbr_pattern]
        address_text = re.sub(abbr_pattern, full_form, address_text)

    # Clean up multiple spaces that might result from replacements
    address_text = re.sub(r'\s+', ' ', address_text).strip()
    return address_text

import pandas as pd
# Separate data by division
df_d1 = main_df[main_df['Division'] == 'D1'].copy()
df_d2 = main_df[main_df['Division'] == 'D2'].copy()

df_d1['ADDR'] = df_d1['CUSTADDR'].apply(standardize_address_abbreviations_enhanced)
df_d2['ADDR'] = df_d2['CUSTADDR'].apply(standardize_address_abbreviations_enhanced)
print(df_d1.head(10))
print(df_d2.head(10))
print(df_d2.describe())
print(df_d1.describe())
#df_standardized.to_csv("standardized_address_regex_based2.csv", index=False)

<>:128: SyntaxWarning: invalid escape sequence '\M'
<>:128: SyntaxWarning: invalid escape sequence '\M'
C:\Windows\Temp\ipykernel_17196\4055270440.py:128: SyntaxWarning: invalid escape sequence '\M'
  main_df=pd.read_csv("D:\ML\grok_adddress_parser\grok_address_parser-env\exp1.csv")


     ID Division                                           CUSTADDR
0   CA1       D1             j 3 118 j laxmi nagar extension 110092
1   CA2       D1             j 3 163 laxmi nagar j extension 110092
2   CA3       D1             f 86 a jawahar park laxmi nagar 110092
3   CA4       D1                           i 186 laxmi nagar 110092
4   CA5       D1  house number 5 195 lalita park laxmi nagar 110092
5   CA6       D1                            i 34 laxmi nagar 110092
6   CA7       D1           10 a vishwakarma park laxmi nagar 110092
7   CA8       D1       18 1 g f vishwakarma park laxmi nagar 110092
8   CA9       D1            7 a vishwakarma park laxmi nagar 110092
9  CA10       D1                         i 173 a laxmi nagar 110092
      ID Division                                  CUSTADDR
73  CA74       D2  harijan basti tukmir pur shahdara 110094
74  CA75       D2           vill tukmir pur shahdara 110094
87  CA88       D2        vill khajoori khas shahdara 110094
88  CA89    

THIS CELL IS THE MAIN NER MODEL INFERENCE CELL , WE GET RAW OUTPUTS WITH CONFIDENCE SCORE ON EACH TOKEN

In [ ]:
from transformers import pipeline
from datasets import Dataset
from tqdm.auto import tqdm
import pandas as pd

print(label_list1)

def ner_batch_runner(df):
    # Prepare HuggingFace Dataset
    dataset = Dataset.from_pandas(df[['ID', 'ADDR','CUSTADDR']])

    # Batch inference
    def run_batch(batch):
        predictions = ner_pipe(batch['ADDR'])
        return {"entities": predictions}

    # Run with batched=True
    dataset = dataset.map(run_batch, batched=True, batch_size=128)

    # Convert back to DataFrame with tqdm progress
    entity_df = dataset.to_pandas()
    return entity_df

# === Run on both D1 and D2 ===
print("Running NER on D1...")
ner_df1 = ner_batch_runner(df_d1)
print(ner_df1.head())
print(ner_df1.describe())
print("Running NER on D2...")
ner_df2 = ner_batch_runner(df_d2)
print(ner_df1.head())
print(ner_df1.describe())
# Optional: Save to file
ner_df1.to_csv("d1_ner_output.csv", index=False)
ner_df2.to_csv("d2_ner_output.csv", index=False)


['B-APARTMENT', 'B-AREA', 'B-BLOCK', 'B-FLOOR', 'B-HOUSE_NUMBER', 'B-KHASRA_NUMBER', 'B-LANDMARK', 'B-LOCALITY', 'B-LOCALITY2', 'B-PINCODE', 'B-PLOT_NUMBER', 'B-ROAD_DETAILS', 'B-VILLAGE', 'I-APARTMENT', 'I-AREA', 'I-BLOCK', 'I-FLOOR', 'I-HOUSE_NUMBER', 'I-KHASRA_NUMBER', 'I-LANDMARK', 'I-LOCALITY', 'I-LOCALITY2', 'I-PLOT_NUMBER', 'I-ROAD_DETAILS', 'I-VILLAGE', 'O']


100%|██████████| 3361/3361 [1:38:40<00:00,  1.76s/it]  


THIS CELL PARSES RAW NER OUTPUT INTO COLOMNS

In [ ]:
from collections import defaultdict

# ✏️ 1) Edit this list if you add / drop labels later
FIELDS = [
    "HOUSE_NUMBER", "PLOT_NUMBER", "FLOOR", "ROAD_DETAILS", "KHASRA_NUMBER",
    "BLOCK", "APARTMENT", "LANDMARK", "LOCALITY", "AREA", "LOCALITY2",
    "VILLAGE", "PINCODE"
]

# 🔧 2) Utility to stitch sub-words: "##no" → "no" (attach to previous token)
def join_word(prev, cur):
    if cur.startswith("##"):
        return prev + cur[2:]
    if prev.endswith("-") or prev.endswith("/"):
        return prev + cur
    return prev + " " + cur if prev else cur

# 🏗️ 3) Convert one NER output list → dict of fields
def convert_single_output(entity_list):
    row = defaultdict(str)

    for ent in entity_list:
        key = ent["entity_group"].upper()
        word = ent["word"]
        if row[key]:
            row[key] = join_word(row[key], word)
        else:
            row[key] = word.lstrip("##")

    # 🌐 Compose complete address (simple comma join)
    ordered_parts = [row[f] for f in FIELDS if row[f]]
    #if row["PINCODE"]:
    #    ordered_parts.append(f"Delhi -{row['PINCODE']}")
    row["complete_address"] = ", ".join(ordered_parts)

    return row

# 🚀 4) Parse both NER outputs into structured format
def parse_ner_output(ner_df):
    parsed_rows = [convert_single_output(ents) for ents in ner_df["entities"]]
    parsed_df = pd.DataFrame(parsed_rows)
    return pd.concat([ner_df[["ID","ADDR"]].reset_index(drop=True), parsed_df], axis=1)
import ast
import re

def fix_entity_column(df):
    def fix_json_str(s):
        if isinstance(s, str):
            # Add missing commas between dictionaries
            s = re.sub(r"\}\s*\{", "}, {", s)
            try:
                return ast.literal_eval(s)
            except Exception as e:
                print("⚠️ Failed to parse:", s)
                return []
        return s
    df["entities"] = df["entities"].apply(fix_json_str)
    return df

#temprory
ner_df1=pd.read_csv("D:\ML\grok_adddress_parser\grok_address_parser-env\d1_ner_output.csv")
ner_df2=pd.read_csv("D:\ML\grok_adddress_parser\grok_address_parser-env\d2_ner_output.csv")
ner_df1 = fix_entity_column(ner_df1)
ner_df2 = fix_entity_column(ner_df2)
# 🗂️ 5) Process D1 and D2
df1 = parse_ner_output(ner_df1)
df2 = parse_ner_output(ner_df2)

# 📝 6) Save parsed addresses
df1 = df1[["ID"] + FIELDS + ["complete_address"]+["CUSTADDR"]]
df2 = df2[["ID"] + FIELDS + ["complete_address"]+["CUSTADDR"]]

df1.to_csv("d1_structured_parsed.csv", index=False)
df2.to_csv("d2_structured_parsed.csv", index=False)

print("✅ Added parsed addresses to structured CSVs.")
print(df1.head())
print(df2.head()) 


✅ Added parsed addresses to structured CSVs.
    ID    HOUSE_NUMBER PLOT_NUMBER FLOOR ROAD_DETAILS KHASRA_NUMBER BLOCK  \
0  CA1             j 3       118 j                                          
1  CA2         j 3 163                          laxmi                       
2  CA3          f 86 a                        jawahar                       
3  CA4           i 186                                                      
4  CA5  house number 5         195                                          

  APARTMENT LANDMARK     LOCALITY               AREA LOCALITY2 VILLAGE  \
0                           nagar    laxmi extension                     
1                                  nagar j extension                     
2                            park        laxmi nagar                     
3                     laxmi nagar                                        
4                     lalita park        laxmi nagar                     

  PINCODE                                   com

---------------------------------------------------------------------------------
CLUSTERING USING EMBEDDINGS ON EXTRACTED SENTENCE using paraphrase-multilingual-MiniLM-L12-v2 AND FAISS 
---------------------------------------------------------------------------------


Global HNSW indexing(graph based clustering technique),here we make indexes on the complete vector db so can be ram intensive on very huge datasets

In [2]:
import os
import json
import numpy as np
import pandas as pd
import faiss
import torch
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from collections import defaultdict, Counter
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
# ─── CONFIG ────────────────────────────────────────────────────────────────────
MODEL_NAME           = 'paraphrase-multilingual-MiniLM-L12-v2'
EMBED_CACHE_DIR      = "embeddings_cache"
GLOBAL_THRESHOLD     = 0.97
GLOBAL_EFSEARCH      = 32#recall
HNSW_M               = 32#neighbouring nodes
HNSW_EFCONSTRUCT     = 80#accuracy and buld time

STOP_WORDS           = ['house number', 'plot number']
# ────────────────────────────────────────────────────────────────────────────────


def embed_and_cache(df, col="address_extract", cache_key="global"):
    os.makedirs(EMBED_CACHE_DIR, exist_ok=True)
    path = f"{EMBED_CACHE_DIR}/{cache_key}.npy"
    if os.path.exists(path):
        print(f"📂 Loading embeddings from cache: {path}")
        return np.load(path)
    print("🧠 Computing embeddings…")
    model = SentenceTransformer(MODEL_NAME,device=device)
    embs = model.encode(df[col].tolist(), show_progress_bar=True, normalize_embeddings=True)
    np.save(path, embs)
    return embs


def build_hnsw(embeddings, M=HNSW_M, efC=HNSW_EFCONSTRUCT):
    dim = embeddings.shape[1]
    idx = faiss.IndexHNSWFlat(dim, M)
    idx.hnsw.efConstruction = efC
    idx.add(embeddings)  # embeddings are already normalized
    return idx


def global_cluster(df, embeddings, threshold=GLOBAL_THRESHOLD, ef_search=GLOBAL_EFSEARCH):
    ids  = df["ID"].tolist()
    addr = df["address_extract"].tolist()
    raw  = df["CUSTADDR"].tolist()

    index = build_hnsw(embeddings)
    index.hnsw.efSearch = ef_search

    k = min(100, len(addr))
    _, neighbors = index.search(embeddings, k)

    visited = set()
    clusters = []
    sim_band_counter = Counter()

    print("🔗 Clustering similar addresses globally...")
    for i, nbrs in tqdm(enumerate(neighbors), total=len(neighbors), desc="Global Clustering"):
        if ids[i] in visited:
            continue

        cluster_idxs = []
        for j in nbrs:
            if j == -1:
                continue
            sim = float(np.dot(embeddings[i], embeddings[j]))
            band = round(sim, 2)
            sim_band_counter[band] += 1

            if sim >= threshold and ids[j] not in visited:
                visited.add(ids[j])
                cluster_idxs.append(j)

        if len(cluster_idxs) > 1:
            clusters.append({
                "cluster_id": f"cluster_{i}",
                "canonical_address": addr[i],
                "members": [
                    {"customer_id": ids[j], "raw_address": raw[j]}
                    for j in cluster_idxs
                ]
            })

    return clusters, sim_band_counter


def mini_cluster_house_plot(clusters, df):
    df_meta = df.set_index("ID")
    enhanced = []

    print("🏠 Creating mini-clusters based on HOUSE_NUMBER and PLOT_NUMBER...")
    for cl in tqdm(clusters, desc="Mini Clustering", total=len(clusters)):
        buckets = defaultdict(list)

        for m in cl["members"]:
            cid = m["customer_id"]
            try:
                h = str(df_meta.at[cid, "HOUSE_NUMBER"]).strip().lower()
                p = str(df_meta.at[cid, "PLOT_NUMBER"]).strip().lower()
            except KeyError:
                continue

            # Clean STOP_WORDS (if any)
            for w in STOP_WORDS:
                h = h.replace(w, "")
                p = p.replace(w, "")
            h, p = h.strip(), p.strip()

            # 🛑 Skip empty both
            if not h and not p:
                continue

            key = (h, p)
            buckets[key].append(m)

        subcls = []
        for (h, p), membs in buckets.items():
            if len(membs) < 2:
                continue
            subcls.append({
                "subcluster_id": f"{cl['cluster_id']}_h{h or 'na'}_p{p or 'na'}",
                "house_number": h or None,
                "plot_number": p or None,
                "members": membs
            })

        cl["mini_clusters"] = subcls
        enhanced.append(cl)

    return enhanced


def report_stats(clusters, sim_counter):
    sizes = [len(c["members"]) for c in clusters]
    return {
        "total_clusters": len(clusters),
        "avg_cluster_size": float(np.mean(sizes)) if sizes else 0,
        "max_cluster_size": int(max(sizes)) if sizes else 0,
        "cluster_size_dist": dict(Counter(sizes)),
        "similarity_band_dist": dict(sim_counter)
    }


d:\conda_envs\address_cluster_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
df2= pd.read_csv('D:/ML/grok_adddress_parser/grok_address_parser-env/d2_structured_parsed.csv')

columns_to_merge = [
    'HOUSE_NUMBER', 'PLOT_NUMBER', 'ROAD_DETAILS', 'LOCALITY',
    'AREA', 'LOCALITY2', 'VILLAGE','APARTMENT'
]

# Combine fields into a structured string
df2['address_extract'] = df2[columns_to_merge].fillna('').apply(lambda row: ', '.join(filter(None, row)), axis=1)
df2['address_extract'] = df2['address_extract'].str.replace(r'\bdelhi\b', '', case=False, regex=True).str.strip(', ').str.strip()
# Fill missing locality values for grouping
df2['LOCCALITY'] = df2['LOCALITY'].fillna('unknown').str.lower().str.strip()
df2['ID'] = df2['ID'].astype(str)
df2["HOUSE_NUMBER"] = df2["HOUSE_NUMBER"].fillna("").astype(str)
df2["PLOT_NUMBER"] = df2["PLOT_NUMBER"].fillna("").astype(str)

#d2_clustering steps
raw_addresses = df2["CUSTADDR"].tolist()
customer_ids  = df2["ID"].tolist()

embs = embed_and_cache(df2, col="address_extract", cache_key="all_addresses_in_d2")
top_clusters,sim_counter = global_cluster(df2, embs, threshold=0.97, ef_search=128)
final = mini_cluster_house_plot(top_clusters, df2)
stats = report_stats(final,sim_counter)
out = {"summary_stats": stats, "clusters": final}

os.makedirs("cluster_output", exist_ok=True)
with open("cluster_output/complete_d2.json", "w") as f:
    json.dump(out, f, indent=2, ensure_ascii=False)

print("✅ Done for d2. Summary:", json.dumps(stats, indent=2))


C:\Windows\Temp\ipykernel_24028\2393011998.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df2= pd.read_csv('D:/ML/grok_adddress_parser/grok_address_parser-env/d2_structured_parsed.csv')


📂 Loading embeddings from cache: embeddings_cache/all_addresses_in_d2.npy
🔗 Clustering similar addresses globally...


Global Clustering: 100%|██████████| 261778/261778 [00:29<00:00, 8815.90it/s] 


🏠 Creating mini-clusters based on HOUSE_NUMBER and PLOT_NUMBER...


Mini Clustering: 100%|██████████| 35596/35596 [00:01<00:00, 31816.24it/s]


✅ Done for d2. Summary: {
  "total_clusters": 35596,
  "avg_cluster_size": 3.562787953702663,
  "max_cluster_size": 100,
  "cluster_size_dist": {
    "2": 20548,
    "27": 21,
    "21": 43,
    "22": 27,
    "3": 6704,
    "6": 915,
    "5": 1637,
    "7": 621,
    "33": 8,
    "11": 170,
    "20": 37,
    "4": 2983,
    "37": 7,
    "29": 16,
    "100": 11,
    "8": 388,
    "9": 307,
    "12": 144,
    "13": 145,
    "18": 40,
    "96": 3,
    "25": 15,
    "36": 8,
    "10": 248,
    "16": 64,
    "15": 77,
    "31": 8,
    "51": 2,
    "77": 1,
    "49": 2,
    "14": 95,
    "24": 17,
    "70": 2,
    "56": 5,
    "19": 35,
    "26": 12,
    "43": 8,
    "23": 19,
    "17": 47,
    "34": 8,
    "95": 1,
    "28": 11,
    "39": 5,
    "98": 3,
    "55": 3,
    "50": 7,
    "67": 2,
    "38": 8,
    "99": 2,
    "35": 14,
    "44": 5,
    "32": 10,
    "83": 2,
    "69": 1,
    "59": 3,
    "62": 2,
    "97": 1,
    "46": 5,
    "68": 4,
    "30": 10,
    "94": 1,
    "47": 4,
    "4

In [2]:
columns_to_merge = [
    'HOUSE_NUMBER', 'PLOT_NUMBER', 'ROAD_DETAILS', 'LOCALITY',
    'AREA', 'LOCALITY2', 'VILLAGE','APARTMENT'
]
#d1 clustering steps
df1= pd.read_csv('D:/ML/grok_adddress_parser/grok_address_parser-env/d1_structured_parsed.csv')
df1['address_extract'] = df1[columns_to_merge].fillna('').apply(lambda row: ', '.join(filter(None, row)), axis=1)
df1['address_extract'] = df1['address_extract'].str.replace(r'\bdelhi\b', '', case=False, regex=True).str.strip(', ').str.strip()
df1['LOCALITY'] = df1['LOCALITY'].fillna('unknown').str.lower().str.strip()
df1['ID'] = df1['ID'].astype(str)
df1["HOUSE_NUMBER"] = df1["HOUSE_NUMBER"].fillna("").astype(str)
df1["PLOT_NUMBER"] = df1["PLOT_NUMBER"].fillna("").astype(str)

raw_addresses = df1["CUSTADDR"].tolist()
customer_ids  = df1["ID"].tolist()

embs = embed_and_cache(df1, col="address_extract", cache_key="all_addresses_in_d1")
top_clusters,sim_counter = global_cluster(df1, embs, threshold=0.97, ef_search=128)
final = mini_cluster_house_plot(top_clusters, df1)
stats = report_stats(final,sim_counter)
out = {"summary_stats": stats, "clusters": final}

os.makedirs("cluster_output", exist_ok=True)
with open("cluster_output/complete_d1.json", "w") as f:
    json.dump(out, f, indent=2, ensure_ascii=False)

print("✅ Done for d1. Summary:", json.dumps(stats, indent=2))


🧠 Computing embeddings…


Batches:   0%|          | 0/6722 [00:00<?, ?it/s]d:\conda_envs\address_cluster_env\lib\site-packages\torch\nn\modules\module.py:1747: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Batches: 100%|██████████| 6722/6722 [01:18<00:00, 85.26it/s] 


: 

PERFORMED SAME INDEXING TECHNIQUE BUT IN BATCHES AND LATER MERGED DIFFERENT CLUSTERS ACROSS BATCHES BASED ON CENTROID SIMILARITY OF THOSE CLUSTERS

CURRENTLY HAS SOME ISSUES WITH RAM USTILIZATION OR DEPENDENCY ISSUES CLUSTERING ALWAYS GETS STUCK AT 35% FOR DATASETS >200K ADDRESSES 

In [1]:
import os
import json
import numpy as np
import pandas as pd
import faiss
import torch
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from collections import defaultdict, Counter
from sklearn.neighbors import NearestNeighbors
import networkx as nx

# ─── CONFIG ────────────────────────────────────────────────────────────────────
MODEL_NAME           = 'paraphrase-multilingual-MiniLM-L12-v2'
EMBED_CACHE_DIR      = "embeddings_cache"
GLOBAL_THRESHOLD     = 0.97
GLOBAL_EFSEARCH      = 20#recall
HNSW_M               = 32#neighbouring nodes
HNSW_EFCONSTRUCT     = 70#accuracy
BATCH_SIZE           = 1000
STOP_WORDS           = ['house number', 'plot number']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ─── FUNCTIONS ──────────────────────────────────────────────────────────────────
def embed_and_cache_batchwise(df, col="address_extract", cache_key="global"):
    os.makedirs(EMBED_CACHE_DIR, exist_ok=True)
    full_embs = []
    model = SentenceTransformer(MODEL_NAME, device=device)
    path = f"{EMBED_CACHE_DIR}/{cache_key}.npy"
    if os.path.exists(path):
        print(f"📂 Loading embeddings from cache: {path}")
        return np.load(path)
    print("🧠 Computing embeddings in batches…")
    for i in tqdm(range(0, len(df), BATCH_SIZE)):
        batch_texts = df[col].iloc[i:i+BATCH_SIZE].tolist()
        embs = model.encode(batch_texts, normalize_embeddings=True)
        full_embs.append(embs)
    full_embs = np.vstack(full_embs)
    np.save(path, full_embs)
    return full_embs

def build_hnsw(embeddings, M=HNSW_M, efC=HNSW_EFCONSTRUCT):
    dim = embeddings.shape[1]
    idx = faiss.IndexHNSWFlat(dim, M)
    idx.hnsw.efConstruction = efC
    idx.add(embeddings)
    return idx

def cluster_batchwise(df, embs, threshold=GLOBAL_THRESHOLD, ef_search=GLOBAL_EFSEARCH):
    all_clusters = []
    all_centroids = []
    id_to_cluster = {}
    id_list = df["ID"].tolist()
    raw_list = df["CUSTADDR"].tolist()
    addr_list = df["address_extract"].tolist()
    sim_counter = Counter()

    for i in tqdm(range(0, len(df), BATCH_SIZE), desc="Batch Clustering"):
        batch_df = df.iloc[i:i+BATCH_SIZE].copy()
        batch_embs = embs[i:i+BATCH_SIZE]
        batch_ids = batch_df["ID"].tolist()
        batch_raw = batch_df["CUSTADDR"].tolist()
        batch_addr = batch_df["address_extract"].tolist()

        index = build_hnsw(batch_embs)
        index.hnsw.efSearch = ef_search
        k = min(100, len(batch_df))
        _, neighbors = index.search(batch_embs, k)

        visited = set()
        for j, nbrs in enumerate(neighbors):
            if batch_ids[j] in visited:
                continue
            cluster_idxs = []
            for n in nbrs:
                if n == -1:
                    continue
                sim = float(np.dot(batch_embs[j], batch_embs[n]))
                sim_counter[round(sim, 2)] += 1
                if sim >= threshold and batch_ids[n] not in visited:
                    visited.add(batch_ids[n])
                    cluster_idxs.append(n)
            if len(cluster_idxs) > 1:
                cid = f"b{i}_cluster_{j}"
                all_clusters.append({
                    "cluster_id": cid,
                    "canonical_address": batch_addr[j],
                    "members": [{"customer_id": batch_ids[k], "raw_address": batch_raw[k]} for k in cluster_idxs]
                })
                centroid = np.mean(batch_embs[cluster_idxs], axis=0)
                all_centroids.append(centroid)
                for k in cluster_idxs:
                    id_to_cluster[batch_ids[k]] = cid

    return all_clusters, np.vstack(all_centroids), sim_counter

def merge_clusters_by_centroids(clusters, centroids, threshold=GLOBAL_THRESHOLD):
    nn = NearestNeighbors(n_neighbors=10, metric='cosine').fit(centroids)
    distances, indices = nn.kneighbors(centroids)
    G = nx.Graph()
    for i, nbrs in enumerate(indices):
        for j, dist in zip(nbrs, distances[i]):
            if dist < (1 - threshold):
                G.add_edge(i, j)
    components = list(nx.connected_components(G))
    merged = []
    for idx, comp in enumerate(components):
        members = []
        for cid in comp:
            members.extend(clusters[cid]["members"])
        if len(members) < 2:
            continue
        merged.append({
            "cluster_id": f"merged_{idx}",
            "canonical_address": members[0]['raw_address'],
            "members": members
        })
    return merged

def mini_cluster_house_plot(clusters, df):
    df_meta = df.set_index("ID")
    enhanced = []
    for cl in tqdm(clusters, desc="Mini Clustering", total=len(clusters)):
        buckets = defaultdict(list)
        for m in cl["members"]:
            cid = m["customer_id"]
            try:
                h = str(df_meta.at[cid, "HOUSE_NUMBER"]).strip().lower()
                p = str(df_meta.at[cid, "PLOT_NUMBER"]).strip().lower()
            except KeyError:
                continue
            for w in STOP_WORDS:
                h = h.replace(w, "")
                p = p.replace(w, "")
            if not h and not p:
                continue
            buckets[(h, p)].append(m)
        subcls = []
        for (h, p), membs in buckets.items():
            if len(membs) < 2:
                continue
            subcls.append({
                "subcluster_id": f"{cl['cluster_id']}_h{h or 'na'}_p{p or 'na'}",
                "house_number": h or None,
                "plot_number": p or None,
                "members": membs
            })
        cl["mini_clusters"] = subcls
        enhanced.append(cl)
    return enhanced

def report_stats(clusters, sim_counter):
    sizes = [len(c["members"]) for c in clusters]
    return {
        "total_clusters": len(clusters),
        "avg_cluster_size": float(np.mean(sizes)) if sizes else 0,
        "max_cluster_size": int(max(sizes)) if sizes else 0,
        "cluster_size_dist": dict(Counter(sizes)),
        "similarity_band_dist": dict(sim_counter)
    }


d:\conda_envs\address_cluster_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
df2= pd.read_csv('D:/ML/grok_adddress_parser/grok_address_parser-env/d2_structured_parsed.csv')

columns_to_merge = [
    'HOUSE_NUMBER', 'PLOT_NUMBER', 'ROAD_DETAILS', 'LOCALITY',
    'AREA', 'LOCALITY2', 'VILLAGE','APARTMENT'
]

# Combine fields into a structured string
df2['address_extract'] = df2[columns_to_merge].fillna('').apply(lambda row: ', '.join(filter(None, row)), axis=1)
df2['address_extract'] = df2['address_extract'].str.replace(r'\bdelhi\b', '', case=False, regex=True).str.strip(', ').str.strip()
# Fill missing locality values for grouping
df2['LOCCALITY'] = df2['LOCALITY'].fillna('unknown').str.lower().str.strip()
df2['ID'] = df2['ID'].astype(str)
df2["HOUSE_NUMBER"] = df2["HOUSE_NUMBER"].fillna("").astype(str)
df2["PLOT_NUMBER"] = df2["PLOT_NUMBER"].fillna("").astype(str)

#d2_clustering steps
raw_addresses = df2["CUSTADDR"].tolist()
customer_ids  = df2["ID"].tolist()
# ... your preprocessing steps here ...
embeddings = embed_and_cache_batchwise(df2, col="address_extract", cache_key="global_d2")
clusters, centroids, sim_counter = cluster_batchwise(df2, embeddings)
print(f"🔗 Batchwise clusters formed: {len(clusters)}")

# Merge clusters via centroid similarity
merged_clusters = merge_clusters_by_centroids(clusters, centroids)
print(f"🧬 Merged clusters: {len(merged_clusters)}")

    # Sub-cluster using house & plot number
final_clusters = mini_cluster_house_plot(merged_clusters, df2)

    # Report stats
stats = report_stats(final_clusters, sim_counter)
print("\n📊 Clustering Summary:")
for k, v in stats.items():
    print(f"{k}: {v}")

    # Save result
with open("cluster_output/complete_d2.json","w") as f:
      json.dump({"summary_stats": stats, "clusters": final_clusters}, f, indent=2, ensure_ascii=False)
print(f"\n✅ Clusters saved to complete_d2.json")


C:\Windows\Temp\ipykernel_18676\1818119027.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df2= pd.read_csv('D:/ML/grok_adddress_parser/grok_address_parser-env/d2_structured_parsed.csv')


📂 Loading embeddings from cache: embeddings_cache/global_d2.npy


Batch Clustering:   7%|▋         | 18/262 [00:02<00:37,  6.47it/s]

: 

In [2]:
columns_to_merge = [
    'HOUSE_NUMBER', 'PLOT_NUMBER', 'ROAD_DETAILS', 'LOCALITY',
    'AREA', 'LOCALITY2', 'VILLAGE','APARTMENT'
]
#d1 clustering steps
df1= pd.read_csv('D:/ML/grok_adddress_parser/grok_address_parser-env/d1_structured_parsed.csv')
df1['address_extract'] = df1[columns_to_merge].fillna('').apply(lambda row: ', '.join(filter(None, row)), axis=1)
df1['address_extract'] = df1['address_extract'].str.replace(r'\bdelhi\b', '', case=False, regex=True).str.strip(', ').str.strip()
df1['LOCALITY'] = df1['LOCALITY'].fillna('unknown').str.lower().str.strip()
df1['ID'] = df1['ID'].astype(str)
df1["HOUSE_NUMBER"] = df1["HOUSE_NUMBER"].fillna("").astype(str)
df1["PLOT_NUMBER"] = df1["PLOT_NUMBER"].fillna("").astype(str)

raw_addresses = df1["CUSTADDR"].tolist()
customer_ids  = df1["ID"].tolist()
embeddings = embed_and_cache_batchwise(df1, col="address_extract", cache_key="global_d1")
clusters, centroids, sim_counter = cluster_batchwise(df1, embeddings)
print(f"🔗 Batchwise clusters formed: {len(clusters)}")

# Merge clusters via centroid similarity
merged_clusters = merge_clusters_by_centroids(clusters, centroids)
print(f"🧬 Merged clusters: {len(merged_clusters)}")

    # Sub-cluster using house & plot number
final_clusters = mini_cluster_house_plot(merged_clusters, df1)

    # Report stats
stats = report_stats(final_clusters, sim_counter)
print("\n📊 Clustering Summary:")
for k, v in stats.items():
    print(f"{k}: {v}")

    # Save result
with open("cluster_output/complete_d1.json","w") as f:
      json.dump({"summary_stats": stats, "clusters": final_clusters}, f, indent=2, ensure_ascii=False)
print(f"\n✅ Clusters saved to complete_d1.json")

📂 Loading embeddings from cache: embeddings_cache/global_d1.npy


Batch Clustering:  14%|█▍        | 30/216 [00:03<00:24,  7.51it/s]

: 

BETTER APPROACH IN FUTURE IS TO USE IVF INDEXING TECHNIQUE 

and probaly perform clustering in a seperate python script as running notebooks adds extra ram and computation overhead